In [1]:
import itertools
import json
import os

import numpy as np
from pathlib import Path
import pandas as pd

In [2]:
BASE_DIR = "../../evaluation/output_evals/emotiontalk"
DIRECTION_PAIRS = ['zh_en']
SYSTEM_NAMES = ['qwen2audio-7b', 'phi4multimodal', 'desta2-8b', 'canary-v2', 'seamlessm4t', 'voxtral-small-24b','whisper']

In [3]:
def load_results_summaries(base_dir, direction_pairs, system_names):
    """
    Loads all result summaries from a directory structure.

    Args:
        base_dir (str or Path): The base directory for the evaluation outputs.
        direction_pairs (list): A list of language direction strings (e.g., 'en_de').
        system_names (list): A list of system name strings.

    Returns:
        dict: A nested dictionary containing the loaded data, structured as
              {direction: {system: [results]}}.
    """
    base_path = Path(base_dir)
    all_results = {}

    # Use itertools.product to cleanly iterate over all combinations
    for direction, system in itertools.product(direction_pairs, system_names):
        summary_path = base_path / system / direction / 'results.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with summary_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]
                
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {summary_path}")
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {summary_path}: {e}")
            all_results[direction][system] = None

    return all_results

In [4]:
def convert_results_to_dataframe(results_data):
    """
    Converts the nested dictionary of results into a single pandas DataFrame.

    Each row corresponds to a single entry, with 'direction' and 'system'
    columns added, and all 'metrics' unpacked into separate columns.
    """
    all_records = []
    for direction, systems in results_data.items():
        for system, records in systems.items():
            if records is None:
                continue
            for record in records:
                # Separate metrics from the record
                metrics = record.pop("metrics", {})  # safely get metrics
                # Merge everything into one flat dict
                flat_record = {
                    "direction": direction,
                    "system": system,
                    **record,
                    **metrics,  # unpack metrics into top-level keys
                }
                all_records.append(flat_record)

    if not all_records:
        print("No records were found to create a DataFrame.")
        return pd.DataFrame()

    df = pd.DataFrame(all_records)

    # Put identifying info up front
    original_cols = [c for c in df.columns if c not in ["direction", "system"]]
    df = df[["direction", "system"] + original_cols]

    return df

In [5]:
def add_emotion_column(df, manifests_dir="../../manifests/emotiontalk"):
    """
    Adds an 'emotion' column to the DataFrame by reading all .jsonl files in the given directory.
    
    Args:
        df (pd.DataFrame): The DataFrame containing at least a 'sample_id' column.
        manifests_dir (str or Path): Directory containing .jsonl manifest files.

    Returns:
        pd.DataFrame: The original DataFrame with a new 'emotion' column.
    """
    manifests_dir = Path(manifests_dir)
    emotion_map = {}

    # Read all .jsonl files in the directory
    for file in manifests_dir.glob("*.jsonl"):
        with open(file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    record = json.loads(line)
                    sid = str(record.get("sample_id"))  # keep as string for safety
                    emo = record.get("benchmark_metadata", {}).get("emotion")
                    if sid and emo:
                        emotion_map[sid] = emo
                except json.JSONDecodeError:
                    continue  # skip bad lines just in case

    if not emotion_map:
        print("No emotion data found in manifest files.")
        df["emotion"] = None
        return df

    # Map the emotion values onto the DataFrame
    df = df.copy()
    df["emotion"] = df["sample_id"].astype(str).map(emotion_map)

    return df

In [6]:
def compute_emotion_strict_scores(df):
    """
    Computes mean metric scores and strict scores grouped by (system, emotion).
    
    Expects columns:
      - system
      - emotion
      - xcomet_qe_score
      - metricx_qe_score
      - linguapy_score (list/tuple of [flag, lang])
    """
    df = df.copy()

    # --- Split linguapy_score into two separate columns ---
    df[["linguapy_flag", "linguapy_lang"]] = pd.DataFrame(
        df["linguapy_score"].tolist(), index=df.index
    )

    # --- Define penalties ---
    penalty_by_metric = {
        "metricx_qe": 25,
        "xcomet_qe": 0,
    }

    # --- Strict score per row ---
    for metric in penalty_by_metric.keys():
        df[f"{metric}_strict"] = df.apply(
            lambda row: row[f"{metric}_score"]
            if row["linguapy_flag"] == 0
            else penalty_by_metric[metric],
            axis=1,
        )

    # --- Aggregate by system × emotion ---
    agg_cols = {
        "linguapy_flag": "mean",  # average from 0–1
    }
    for metric in penalty_by_metric.keys():
        agg_cols[f"{metric}_score"] = "mean"
        agg_cols[f"{metric}_strict"] = "mean"

    result = (
        df.groupby(["system", "emotion"])
        .agg(agg_cols)
        .reset_index()
        .rename(columns={"linguapy_flag": "linguapy_avg"})
    )

    result['linguapy_avg'] = result['linguapy_avg']*100
    return result

In [7]:
results_full = load_results_summaries(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)

In [8]:
df=convert_results_to_dataframe(results_full)

In [9]:
#Need to add the column for emotion ID
df = add_emotion_column(df)

In [10]:
col_map = {
    "linguapy_avg":"LinguaPy",
    "metricx_qe_strict":"QEMetricX_24-Strict-linguapy",
    "xcomet_qe_strict": "XCOMET-QE-Strict-linguapy"
}

#Collapse and get the metrics balanced by the linguapy score
for pair in DIRECTION_PAIRS:
    sub_df = df[df['direction']==pair]
    sub_df = compute_emotion_strict_scores(sub_df)
    #Standardize col names
    sub_df = sub_df.rename(columns=col_map)
    #Save 
    sub_df.to_csv(f"emotiontalk_{pair}.csv",index=False)

In [11]:
sub_df

,system,emotion,LinguaPy,metricx_qe_score,QEMetricX_24-Strict-linguapy,xcomet_qe_score,XCOMET-QE-Strict-linguapy
0,desta2-8b,angry,0.579710,5.508873,5.620458,0.636206,0.634996
1,desta2-8b,disgusted,2.189781,5.810397,6.295648,0.560489,0.556512
2,desta2-8b,fearful,3.370787,5.546735,6.266307,0.503794,0.492294
3,desta2-8b,happy,3.389831,4.867989,5.557113,0.582468,0.569638
4,desta2-8b,neutral,4.872881,4.713634,5.756562,0.626268,0.603019
5,desta2-8b,sad,3.191489,3.817049,4.372170,0.729603,0.717396
6,desta2-8b,surprised,6.293706,5.012291,6.314299,0.550876,0.529782
7,phi4multimodal,angry,7.536232,4.445933,5.954566,0.755909,0.699046
8,phi4multimodal,disgusted,8.029197,5.201147,6.675482,0.717819,0.661363
9,phi4multimodal,fearful,8.988764,4.697793,6.510982,0.649054,0.624152


In [14]:
df.emotion.value_counts()

emotion
neutral      5664
angry        2070
happy        1062
surprised     858
disgusted     822
sad           564
fearful       534
Name: count, dtype: int64